## Coroutines

    - pep-0342
    - generators are data producers
    - coroutines are data consumers

    - Coroutines consume values using a (yield)

In [1]:
def hello():  # -- generator
    # print('Hello world')
    # return 'Hello world'
    yield "Hello world"


hello()

<generator object hello at 0x7f61d0801000>

In [2]:
result = hello()
print(f"{type(result)} {result}")  # generator
print(f"{next(result) = }")  # 'Hello world'

<class 'generator'> <generator object hello at 0x7f61d08010e0>
next(result) = 'Hello world'


In [3]:
def hello():  # generator
    value = "Hello world"
    yield value


hello()

<generator object hello at 0x7f61d0800a50>

In [4]:
result = hello()
print(f"{type(result)} {result}")  # generator
print(f"{next(result) = }")  # 'Hello world'

<class 'generator'> <generator object hello at 0x7f61d0801070>
next(result) = 'Hello world'


In [5]:
def hello():  # coroutine
    value = yield "Hello world"
    yield value


result = hello()
print(f"{type(result)} {result}")  # generator
print(f"{next(result) = }")  # 'Hello world'

<class 'generator'> <generator object hello at 0x7f61d0800d60>
next(result) = 'Hello world'


In [6]:
def hello():  # coroutine
    value = yield
    yield value


# NOTE: Default initial yield is None

result = hello()
print(f"{type(result)} {result}")
print(f"{next(result) = }")  # None

<class 'generator'> <generator object hello at 0x7f61d0801700>
next(result) = None


# # Anothr Examples

In [7]:
def my_coroutine(num):
    value = yield "default string"
    yield value
    yield value
    value = yield "new string"
    yield value

In [8]:
# Step 1: To call
c = my_coroutine(4)
print(f"{type(c)    = }")  # <class 'generator'>
print(f"{c          = }")  # <generator object my_coroutine at 0x0000022D8B349A10>

type(c)    = <class 'generator'>
c          = <generator object my_coroutine at 0x7f61d0801850>


In [9]:
# Step 2: to prime the generator to become coroutine
print(f"{next(c) =}")  # 'default string'

next(c) ='default string'


In [10]:
# Step 3: Sending values to coroutine
print(f"{c.send('first')  =}")  # 'first'
print(f"{c.send('second') =}")  # 'first'
print(f"{c.send('third')  =}")  # 'new string'

c.send('first')  ='first'
c.send('second') ='first'
c.send('third')  ='new string'


In [11]:
c.send("fourth")

'fourth'

In [13]:
print(f"{c.send('first')  =}")  

StopIteration: 

In [14]:
# To close the coroutine
c.close()

## Anoother Example with Infiite receive

In [15]:
def my_coroutine():
    while True:
        received = yield 1234  # values will be received here
        print(f"Received :{received}")


# Step 1: creating the generator
it = my_coroutine()

# Step 2: Prime the coroutine
print(f"{next(it) =}")  # 1234


next(it) =1234


In [16]:
# Step 3: sending values to coroutine
it.send("First")

Received :First


1234

In [17]:
it.send("Second")
it.send("third")

Received :Second
Received :third


1234

In [18]:
for i in range(9):
    it.send(i)

Received :0
Received :1
Received :2
Received :3
Received :4
Received :5
Received :6
Received :7
Received :8


In [19]:
# Step 4: close the coroutine
it.close()


In [20]:
try:
    it.send("fourth")
except StopIteration:
    print("coroutine is closed. Cant send any value")

coroutine is closed. Cant send any value


### Finite Rception

In [21]:
def my_coroutine(count=5):
    for _ in range(count):
        received = yield 1234  # values will be received here
        print(f"Received :{received}")

# Step 1: creating the generator
it = my_coroutine(5)

# Step 2: Prime the coroutine
print(f"{next(it) =}")


# Step 3: sending values to coroutine
it.send("First")
it.send("Second")
it.send("third")


next(it) =1234
Received :First
Received :Second
Received :third


1234

In [22]:
for i in range(9):
    it.send(i)


Received :0
Received :1


StopIteration: 

In [23]:
# Step 4: close the coroutine
it.close()

try:
    it.send("fourth")
except StopIteration:
    print("coroutine is closed. Cant send any value")


coroutine is closed. Cant send any value


##  chaining the coroutines

In [ ]:
def producer(sentence, next_coroutine):
    """
    Producer which just split strings and
    feed it to pattern_filter coroutine
    """
    tokens = sentence.split(" ")
    for token in tokens:
        next_coroutine.send(token)
    next_coroutine.close()


def pattern_filter(pattern="ing", next_coroutine=None):
    """
    Search for pattern in received token
    and if pattern got matched, send it to
    print_token() coroutine for printing
    """
    print("Searching for {}".format(pattern))
    try:
        while True:
            token = yield
            if pattern in token:
                next_coroutine.send(token)
    except GeneratorExit:
        print("Done with filtering!!!")

